In [1]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [2]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

errorIndication: None
errorStatus: 0
errorIndex: 0
varBinds: [ObjectType(ObjectIdentity(ObjectName('1.3.6.1.2.1.1.1.0')), DisplayString(b'Linux li744-109 4.1.5-x86_64-linode61 #7 SMP Mon Aug 24 13:46:31 EDT 2015 x86_64', subtypeSpec=ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(ConstraintsIntersection(), ValueSizeConstraint(0, 65535)), ValueSizeConstraint(0, 255)), ValueSizeConstraint(0, 255))))]


chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

## 同步

In [9]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)



errorIndication, errorStatus, errorIndex, varBinds = cmdGen.nextCmd(
    cmdgen.CommunityData('Huawei@123'),
    cmdgen.UdpTransportTarget(('10.33.131.2', 161)),
    cmdgen.MibVariable('IF-MIB', 'ifOperStatus')
)


print(errorIndication)
print(errorStatus)
print(errorIndex)
print('*' * 50)

for row in varBinds:
    for oid, val in row:
        print(oid.getMibSymbol()[-1][0], val.prettyPrint())

No SNMP response received before timeout
0
0
**************************************************


ValueError: too many values to unpack (expected 2)

## 异步

In [10]:
import string
# from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
# cmdGen = cmdgen.CommandGenerator()  # 同步
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

# mibBuilder = builder.MibBuilder()
# mibBuilder.loadModules(
#     'RFC-1213', 'RFC-3413'
# )
# mibView = view.MibViewController(mibBuilder)


mib_vars = (
    {'desc': 'ifOperStatus', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOperStatus')},  # 端口状态
#     {'desc': 'ifSpeed', 'mib': cmdgen.MibVariable('IF-MIB', 'ifSpeed')},  # 端口速度 bits/s

)

ttt = []
def callback(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    
    for row in varBinds:
        for oid, val in row:
#             if cbCtx['object_name'] not in oid.prettyPrint():
#                 return 0
            
            print(oid.prettyPrint(), val.prettyPrint())
            ttt.append(oid)
#             return
            
    return 1  # 告诉 walk 继续运行
            


for mib_var in mib_vars:
    cbCtx = {'object_name': mib_var['desc']}
    cmdGen.nextCmd(
        cmdgen.CommunityData('huawei@123'),
        cmdgen.UdpTransportTarget(('10.32.33.6', 161)),
        (mib_var['mib'], cmdgen.MibVariable('.1.3.6.1.2.1.1.3')),
        (callback, cbCtx),
        True, True
    )

    
# 启动
cmdGen.snmpEngine.transportDispatcher.runDispatcher()

IF-MIB::ifOperStatus b''
SNMPv2-MIB::sysUpTime b''


In [39]:
t = ttt[2]
int(t.getMibSymbol()[-1][0])

2